<a href="https://colab.research.google.com/github/SupunTSW/Mastering-PyTorch/blob/main/06_Optimization_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Optimization Model Parameters

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size = 64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26.4M/26.4M [00:01<00:00, 19.1MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 303kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.51MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.28MB/s]


##Hyperparameters

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

##Optimization Loop

In [5]:
#Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [6]:
#Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate )

##Full Implementation

In [7]:
def train_loop(dataLoader, model, loss_fn, optimizer):
  size = len(dataLoader.dataset)

  model.train()
  for batch, (x,y) in enumerate(dataLoader):
    #compute prediction and loss
    pred = model(x)
    loss = loss_fn(pred,y)

    #Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * batch_size + len(x)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataLoader, model, loss_fn):
  model.eval()
  size = len(dataLoader.dataset)
  num_batches = len(dataLoader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for x,y in dataLoader:
      pred = model(x)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------
loss: 2.308036 [   64/60000]
loss: 2.286292 [ 6464/60000]
loss: 2.268705 [12864/60000]
loss: 2.265753 [19264/60000]
loss: 2.236603 [25664/60000]
loss: 2.213455 [32064/60000]
loss: 2.218605 [38464/60000]
loss: 2.187846 [44864/60000]
loss: 2.195170 [51264/60000]
loss: 2.149143 [57664/60000]
Test Error: 
 Accuracy: 47.8%, Avg loss: 2.146274 

Epoch 2
------------------------------
loss: 2.162416 [   64/60000]
loss: 2.147586 [ 6464/60000]
loss: 2.086709 [12864/60000]
loss: 2.110264 [19264/60000]
loss: 2.050872 [25664/60000]
loss: 1.983723 [32064/60000]
loss: 2.022713 [38464/60000]
loss: 1.939992 [44864/60000]
loss: 1.958738 [51264/60000]
loss: 1.869259 [57664/60000]
Test Error: 
 Accuracy: 56.7%, Avg loss: 1.871904 

Epoch 3
------------------------------
loss: 1.910451 [   64/60000]
loss: 1.875000 [ 6464/60000]
loss: 1.753874 [12864/60000]
loss: 1.807412 [19264/60000]
loss: 1.686326 [25664/60000]
loss: 1.624380 [32064/60000]
loss: 1.664809 [38464/600